In [15]:
import numpy as np
from numpy.random import permutation
import keras
from keras.utils import np_utils
from keras.models import Model, Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Merge
from keras.optimizers import SGD, RMSprop
from keras import backend
from keras.layers import Input, merge
from keras.layers.core import Dense, Lambda, Reshape
from keras.layers.convolutional import Conv1D
from keras.models import Model

In [16]:
batch_size = 128
nb_epoch = 400
J = 0
#J = 4 #nb of negative targets
L = 32 #for last Dense layers

In [17]:
# For training
genes_1 = np.load('input10k-7gram_gene_compl.npy') # 100 000 pairs from the head of "input_for_neuronet
mirs_1 = np.load('input10k-7gram_mir_compl.npy')

In [18]:

# Generation of 4 'negative' genes by simply shuffling the original genes' set
perm = permutation(len(genes_1))
genes_01 = genes_1[perm]
print perm, genes_01
perm = permutation(len(genes_1))
genes_02 = genes_1[perm]
perm = permutation(len(genes_1))
genes_03 = genes_1[perm]
perm = permutation(len(genes_1))
genes_04 = genes_1[perm]

[9371 1339 2833 ..., 8718 2732 9870] [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [19]:
genes = np.concatenate((genes_1, genes_01, genes_02, genes_03
                        , genes_04))
targets_1 = np.empty(genes_1.shape[0])
targets_1.fill(1) 
targets_0 = np.zeros(4*genes_1.shape[0])
targets = np.concatenate((targets_1, targets_0))
mirs = np.concatenate((mirs_1, mirs_1, mirs_1, mirs_1
                       , mirs_1))

perm = permutation(len(genes))
genes = genes[perm]
mirs = mirs[perm]
targets = targets[perm]

In [20]:
# For test
genes_1t = np.load('input10ktail-7gram_mir_compl.npy') # 10 000 pairs from the tail
mirs_1t = np.load('input10ktail-7gram_gene_compl.npy')
perm = permutation(len(genes_1t))
genes_01t = genes_1[perm]
perm = permutation(len(genes_1t))
genes_02t = genes_1[perm]
perm = permutation(len(genes_1t))
genes_03t = genes_1[perm]
#perm = permutation(len(genes_1t))
#genes_04t = genes_1[perm]

genes_t = np.concatenate((genes_1t, genes_01t, genes_02t, genes_03t))
                          #, genes_04t))
targets_1t = np.empty(genes_1t.shape[0])
targets_1t.fill(1) 
targets_0t = np.zeros(3*genes_1t.shape[0])
targets_t = np.concatenate((targets_1t, targets_0t))
mirs_t = np.concatenate((mirs_1t, mirs_1t, mirs_1t, mirs_1t))
                         #, mirs_1t))

perm_t = permutation(len(genes_t))
genes_t = genes_t[perm_t]
mirs_t = mirs_t[perm_t]
targets_t = targets_t[perm_t]

In [7]:
dense1 = 32
dense2 = 32
dense3 = 1

# targets 
genes_branch = Sequential()
genes_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
#genes_branch.add(Dense(dense2))
#genes_branch.add(Activation('relu'))
#genes_branch.add(Dropout(0.5))
#genes_branch.add(Dense(dense3))
#genes_branch.add(Activation('sigmoid'))

# microRNAs
mirs_branch = Sequential()
mirs_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
#mirs_branch.add(Dense(dense2))
#mirs_branch.add(Activation('relu'))
#mirs_branch.add(Dropout(0.5))
#mirs_branch.add(Dense(dense3))
#mirs_branch.add(Activation('sigmoid'))

In [8]:
# R_Q_D= the cosine similarity between microRNAs and genes
R_Q_D = Merge([mirs_branch, genes_branch], mode = 'concat') # See equation (4).
model_Rs = Sequential()
#model_Rs.add(concat_Rs)
model_Rs.add(R_Q_D)
model_Rs.add(Dense(1))
model_Rs.add(Activation('sigmoid'))
print model_Rs.output_shape
#model_Rs.add(Reshape((J + 1, 1)))
print model_Rs.output_shape
#weight = np.array([1]).reshape(1, 1, 1,1)
#model_Rs.add(Conv1D(1, 1, border_mode = "same", 
 #                    input_shape = (J + 1, 1), 
#                  activation = "linear", bias = False,
#                       weights = [weight]))
#model_Rs.add(Reshape((J + 1, )))
#print model_Rs.output_shape

(None, 1)
(None, 1)


In [9]:
sgd = SGD(lr=0.00015, decay=1e-6, momentum=0.7, nesterov=True)
#model_Rs.add(Lambda(lambda x: backend.softmax(x), output_shape = (J + 1, )))
model_Rs.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics=['accuracy'])

In [10]:
model_Rs.fit([mirs, genes], targets,
             batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_split=0.1)
            #shuffle = True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/400
45000/45000 [==============================] - 2s - loss: 0.5194 - acc: 0.7990 - val_loss: 0.5104 - val_acc: 0.7972
Epoch 2/400
45000/45000 [==============================] - 0s - loss: 0.5093 - acc: 0.8003 - val_loss: 0.5109 - val_acc: 0.7972
Epoch 3/400
45000/45000 [==============================] - 1s - loss: 0.5085 - acc: 0.8003 - val_loss: 0.5091 - val_acc: 0.7972
Epoch 4/400
45000/45000 [==============================] - 1s - loss: 0.5068 - acc: 0.8003 - val_loss: 0.5093 - val_acc: 0.7972
Epoch 5/400
45000/45000 [==============================] - 1s - loss: 0.5066 - acc: 0.8003 - val_loss: 0.5088 - val_acc: 0.7972
Epoch 6/400
45000/45000 [==============================] - 1s - loss: 0.5054 - acc: 0.8003 - val_loss: 0.5081 - val_acc: 0.7972
Epoch 7/400
45000/45000 [==============================] - 1s - loss: 0.5051 - acc: 0.8003 - val_loss: 0.5086 - val_acc: 0.7972
Epoch 8/400
45000/45000 [==============================

In [ ]:
# test
model_Rs.evaluate([mirs_t, genes_t], targets_t, batch_size=batch_size)

39168/40000 [============================>.] - ETA: 0s

[1.1374808326721191, 0.75]

In [ ]:
genes_branch = Sequential()
genes_branch.add(Dense(128, input_dim=genes.shape[1]))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))

mirs_branch = Sequential()
mirs_branch.add(Dense(128, input_dim=genes.shape[1]))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))

merged = Merge([genes_branch, mirs_branch], mode='concat') #Multiple Sequential instances can be merged into a 
# single output via a Merge layer. The output is a layer 
model = Sequential()
model.add(merged)
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

batch_size = 256
nb_epoch = 400
sgd = SGD(lr=0.0002, decay=1e-6, momentum=0.8, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit([genes, mirs], targets,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_split=0.1,
              shuffle=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/400
45000/45000 [==============================] - 1s - loss: 0.6851 - acc: 0.6443 - val_loss: 0.5421 - val_acc: 0.8030
Epoch 2/400
45000/45000 [==============================] - 1s - loss: 0.6148 - acc: 0.7310 - val_loss: 0.5335 - val_acc: 0.8042
Epoch 3/400
45000/45000 [==============================] - 1s - loss: 0.5976 - acc: 0.7558 - val_loss: 0.5301 - val_acc: 0.8042
Epoch 4/400
45000/45000 [==============================] - 1s - loss: 0.5887 - acc: 0.7643 - val_loss: 0.5271 - val_acc: 0.8042
Epoch 5/400
45000/45000 [==============================] - 1s - loss: 0.5858 - acc: 0.7676 - val_loss: 0.5243 - val_acc: 0.8042
Epoch 6/400
45000/45000 [==============================] - 1s - loss: 0.5807 - acc: 0.7723 - val_loss: 0.5220 - val_acc: 0.8042
Epoch 7/400
45000/45000 [==============================] - 1s - loss: 0.5728 - acc: 0.7772 - val_loss: 0.5199 - val_acc: 0.8042
Epoch 8/400
45000/45000 [==============================

In [ ]:
# test
model.evaluate([mirs_t, genes_t], targets_t, batch_size=batch_size)

In [ ]:
dense1 = 128
dense2 = 32
dense3 = 1

# targets 
genes_branch = Sequential()
genes_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
genes_branch.add(Dense(dense2))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
#genes_branch.add(Dense(dense3))
#genes_branch.add(Activation('sigmoid'))

# microRNAs
mirs_branch = Sequential()
mirs_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
mirs_branch.add(Dense(dense2))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
#mirs_branch.add(Dense(dense3))
#mirs_branch.add(Activation('sigmoid'))

R_Q_D = Merge([mirs_branch, genes_branch], mode = 'concat') # See equation (4).
model_Rs = Sequential()
#model_Rs.add(concat_Rs)
model_Rs.add(R_Q_D)
model_Rs.add(Dense(1))
model_Rs.add(Activation('sigmoid'))
print model_Rs.output_shape

sgd = SGD(lr=0.00015, decay=1e-6, momentum=0.8, nesterov=True)
#model_Rs.add(Lambda(lambda x: backend.softmax(x), output_shape = (J + 1, )))
model_Rs.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics=['accuracy'])

model_Rs.fit([mirs, genes], targets,
             batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_split=0.1)

In [ ]:
# test
model_Rs.evaluate([mirs_t, genes_t], targets_t, batch_size=batch_size)